# Aprendizaje Supervisado I
# UdeSA, 2023
## Profesores: Marcela Svarc, Juan Manuel Pérez


## Taller 1: Algoritmos de clasificación + evaluación


**Fecha de entrega: 05/10/2023, 23:59 hs**

Enviar individualmente a perezj@udesa.edu.ar con subject [TALLER-SUPERVISADO] <APELLIDO>



Este notebook contiene los ejercicios del taller 1. Los mismos deberán ser realizados usando Python. Pueden hacerlo en colab como en sus propias computadoras.



In [4]:
import sklearn
from sklearn.metrics import accuracy_score
import numpy as np

# 1. Métricas

### 1.1 Programar las siguientes métricas como funciones de python con la siguiente aridad:


```python
def metrica(y_true,y_pred):
  ...
  return score
```



1.   accuracy
2.   precision
3.   recall
4.   F1-score
5.   F $_{\beta}$-score
6.   Matriz de confusión


In [44]:
def accuracy(y_true, y_pred):
    x=(y_true==y_pred)
    return x.mean()

def precision(y_true, y_pred):
    tp=(y_true==y_pred) & (y_pred==1)
    tp=tp.sum()
    fp=(y_true==0) & (y_pred==1)
    fp=fp.sum()
    pr=tp/(tp+fp)
    return pr

def recall(y_true, y_pred):
    tp=(y_true==y_pred) & (y_pred==1)
    tp=tp.sum()
    fn=(y_true==1) & (y_pred==0)
    fn=fn.sum()
    rc=tp/(tp+fn)
    return rc

def f1(y_true, y_pred):
    nominador=precision(y_true, y_pred)*recall(y_true, y_pred)
    denominador=precision(y_true, y_pred)+recall(y_true, y_pred)
    ef1=(2*nominador)/denominador
    return ef1

def confusion_matrix(y_true, y_pred):
    tp=(y_true==y_pred) & (y_pred==1)
    tp=tp.sum()
    fp=(y_true==0) & (y_pred==1)
    fp=fp.sum()
    fn=(y_true==1) & (y_pred==0)
    fn=fn.sum()
    matrix=np.array([[tp, fp],[fn,tp]])  
    return matrix

confusion_matrix(y_true, y_pred)

array([[4, 2],
       [2, 4]])

### 1.2 Crear diversos casos de test que prueben diferentes aspectos de cada metrica

In [45]:
"""
TODO: Crear tests usando asserts
"""

y_true = np.array([1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1])
y_pred = np.array([1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1])

assert np.isclose(accuracy(y_true, y_pred), 7/10)

AssertionError: 

## Clasificadores

Vamos a implementar los clasificadores usando el mismo esquema de la librería `sklearn`. Para ello, vamos a crear una clase `Clasificador` que tenga los siguientes métodos:

*   `fit(X,y)`: recibe los datos de entrenamiento y los labels y entrena el clasificador
*   `predict(X)`: recibe los datos de test y devuelve las predicciones

Empecemos con clasificadores baseline, es decir, clasificadores que no aprenden nada o casi nada de los datos

### 1.3 Random

Implementar un clasificador que prediga una clase al azar. Para ello, usar la función `np.random.randint`

In [2]:
class RandomClassifier:
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def fit(self, X, y):
        """
        TODO: Completar
        """
        pass

    def predict(self, X):
        """
        TODO: Completar
        X va a ser un np.array de tamaño (n,p)
        n entradas y p predictores
        Que queremos devolver? un np.array de tamaño n
        """
        n=X.shape[0]
        return np.random.randint(low=0,high=self.num_classes, size=n)

In [6]:
# Prueba: genero 1000 etiquetas con 0 y 1000 etiquetas con 1
# Un clasificador aleatorio debería tener un accuracy cercano a 0.5 (¿por qué?)
X=np.random.randint(0,2,size=(200,2))
y_true = np.concatenate([np.zeros(1000), np.ones(1000)])

clf=RandomClassifier(2)

clf.fit(X, y_true)

y_pred=clf.predict(X)
y_pred = RandomClassifier(2).predict(y_true)
assert np.isclose(accuracy_score(y_true, y_pred), 0.5, atol=0.05)

## 1.4 Majority Class Classifier

Programar un algoritmo que prediga la clase mayoritaria sobre los datos



Mirar la documentación de `np.bincount` y `np.argmax`. ¿Para qué sirven?

In [42]:
y = np.array([1,0,0,2,2,2,3,3,4,4,4,4,4,4,4,4])
c=np.bincount(y)
print(c)
np.argmax(c)

[2 1 3 2 8]


4

In [47]:
# el majority classifier es una combinacion de bincount y argmax
class MajorityClassifier:
    def __init__(self):
        self.majority = None

    def fit(self, X, y):
        c=np.bincount(y)
        self.mc=np.argmax(c)


    def predict(self, X):
        return(self.mc)


In [48]:
y=np.array([1,2,2,3,4])
X=np.array([0,0,0,1,1])
modelo=MajorityClassifier()
modelo=modelo.fit(X,y)
predict=modelo.predict(X)
predict

AttributeError: 'NoneType' object has no attribute 'predict'

## 2. Naive Bayes

Supongamos que tenemos un problema de clasificación con $K$ clases, y $p$ predictores/features.

Recordemos que por el teorema de Bayes, podemos escribir la probabilidad de que una instancia $x$ pertenezca a una clase $k$ (posterior) como:

$$P(y=k|x) = \frac{P(x|y=k)P(y=k)}{P(x)}$$

- $P(x|y=k)$ es la probabilidad condicional de $x$ dado que la clase es $k$. Podemos escribirla como $f_k(x)$, donde $f_k$ es la función de densidad de probabilidad de la clase $k$.
- $P(y=k)$ lo podemos modelar como la frecuencia de la clase $k$ en el conjunto de entrenamiento (usualmente, escrita como $\pi_k$)
- $P(x)$ es la probabilidad marginal de $x$, que podemos calcular como la suma de las probabilidades condicionales de $x$ para cada clase $k$ (Probabilidad total)
$$P(x) = \sum_{k=1}^K P(x|y=k)P(y=k)$$

Reescribiendo todo esto, podemos calcular la probabilidad de que una instancia $x$ pertenezca a una clase $k$ como:

$$P(y=k|x) = \frac{f_k(x)\pi_k}{\sum_{j=1}^K f_j(x)\pi_j}$$

$x$ es un vector de $p$ dimensiones, y cada una de sus componentes es una variable aleatoria. Si asumimos que cada componente de $x$ es independiente de las demás, podemos escribir la función de densidad de probabilidad de la clase $k$ como:

$$f_k(x) = \prod_{j=1}^p f_{kj}(x_j)$$

Entonces, para cada clase $k$, tenemos que estimar $p$ funciones de densidad de probabilidad $f_{kj}(x_j)$, una para cada predictor $j$ (tenemos $K \times p$ predictores).

## 3.1 Precalentamiento

### 3.1.2 Estimar funcion de densidad para variables categóricas

Sea $X$ un vector de observaciones de una variable Bernoulli. Utilizar `np.bincount` para estimar la función de densidad de probabilidad de $X$.

In [ ]:
X = np.array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1])

# TODO: Completar


Utilizar `scipy.stats.rv_discrete` para estimar la función de densidad de probabilidad de $X$

La función recibe un parámetro `values` que toma un par de arrays `(Xk, pk)` donde `Xk` son los valores que toma la variable aleatoria y `pk` son las probabilidades asociadas a cada valor.

Utilizar la función `.pmf` para evaluar la función de densidad de probabilidad en un punto.

In [ ]:
import scipy

probas = np.bincount(X) / X.shape[0]

values = np.unique(X)

F = scipy.stats.rv_discrete(values=(values, probas))

F.pmf(0), F.pmf(1), F.pmf(2)

### 3.1 Programar naive bayes para variables categoricas


In [ ]:
import scipy

class NaiveBayes:
    """
    Naive bayes para variables categóricas
    """
    def __init__(self):
        self.priors = None
        self.densities = None
        self.classes = None

    def fit(self, X, y):
        """
        Entrenar Naive Bayes categórico

        Argumentos:
        ----------

        X: array de numpy de forma (n_samples, n_features)
            Datos de entrenamiento. Cada feature debe ser una variable categórica
        y: array de variables categóricas
            Etiquetas de las clases
        """

        assert X.dtype == int, "X debe ser int"

        """
        Una pequeña ayudita para entender el código:

        Hay que calcular dos cosas:

        - Las probabilidades a priori de cada clase
        - Las funciones de densidad de cada variable categórica X_1, ... X_n para cada clase

        """

        # TODO: Calcular prior
        self.priors = np.bincount(y) / len(y)
        # TODO: calcular cantidad de clases (usar np.unique)
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        num_features = X.shape[1]



        # TODO: Calcular Las funciones de densidad de cada variable categórica X_1, ... X_n para cada clase
        # Armo una matriz de n_classes x num_features que sería la cantidad de f_ij que necesito
        self.densities = [[None] * num_features for i in range(n_classes)]

        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            """
            TODO: Completar
            """


    def predict(self, X):
        """
        Predecir las etiquetas de las clases para cada dato de X
        """
        assert self.priors is not None, "Hay que llamar a fit primero"

        """
        TODO: Completar
        """




Vamos a probar todo esto en un dataset generado aleatoriamente. Este dataset consta de 200 observaciones con 5 variables Bernoulli $X_1, X_2, X_3, X_4, X_5$

$$Y|X_1, X_2, X_3, X_4, X_5 \sim Bernoulli(\frac{X_1 + \ldots + X_5}{5})$$

In [ ]:

n_samples = 200
num_vars = 5

np.random.seed(2023)

X = np.random.randint(0, 2, (n_samples, num_vars))

# Creamos  Bernoulli(P/num_vars)

P = X.sum(axis=1) / num_vars
y = np.random.binomial(1, P)

# Train / test split

X_train, y_train = X[:160], y[:160]
X_test, y_test = X[160:], y[160:]

Entrenar el modelo sobre los datos de entrenamiento y predecir sobre los datos de test para:

- RandomClassifier
- MajorityClassifier
- NaiveBayes

Reportar accuracy, precision, recall y F1-score para cada clasificador



- ¿Anda mejor Naive Bayes que los clasificadores baseline? ¿se cumplen las hipótesis de Naive Bayes en este caso?

- **OPCIONAL**. Calcular el error de Bayes. ¿Cómo se compara con el de Naive Bayes?

In [ ]:
# Calculate correlation between the sum and Y

clf = NaiveBayes()

"""
TODO: Completar
"""


In [ ]:
clf = RandomClassifier(2)


"""
TODO: Completar
"""

In [ ]:
clf = MajorityClassifier()


"""
TODO: Completar
"""


### 3.2 Programar naive bayes para variables gaussianas.
¿Cómo podria estimar $P(a_i|v_j)$? Piense en fitear diferentes distribuciones que le permitan luego inferir valores nuevos.

Ayuda: Puede usera `scipy.stats.norm( mu , std ).pdf( new_val )` como funcion de densidad de probabilidad de una normal y evaluarla en new_val




In [ ]:
class GaussianNaiveBayes:
    """
    Naive bayes para variables gaussianas
    """
    def __init__(self):
        self.priors = None
        self.densities = None
        self.classes = None

    def fit(self, X, y):
        """
        Entrenar Naive Bayes gaussiano

        Argumentos:
        ----------

        X: array de numpy de forma (n_samples, n_features)
            Datos de entrenamiento. Cada feature debe ser una variable categórica
        y: array de variables categóricas
            Etiquetas de las clases
        """


        """
        Una pequeña ayudita para entender el código:

        Hay que calcular dos cosas:

        - Las probabilidades a priori de cada clase
        - Las funciones de densidad de cada variable categórica X_1, ... X_n para cada clase

        """

        """
        TODO: Completar
        """

    def predict(self, X):
        """
        Predict class for each sample in X

        Arguments:
        ----------

        X: numpy array of shape (n_samples, n_features)
            Training data. Each feature must be a categorical variable

        Returns:
        --------

        y_pred: numpy array of shape (n_samples,)
            Predicted class for each sample
        """
        assert self.priors is not None, "You must call fit before predict"



        """
        TODO: Completar
        """


### 3.3 LDA

Programar LDA para variables gaussianas

Utilizar las funciones `np.mean` y `np.cov` para estimar los parámetros de las gaussianas

In [ ]:
class LDAClassifier:
    def __init__(self):
        self.distributions = None
        self.priors = None
        self.classes = None

    def fit(self, X, y):
        """
        TODO: Completar
        """


    def predict(self, X):
        """
        TODO: Completar
        """


#### Opcional: Implementar QDA

Leer en el ISLP el capítulo de QDA y programar el algoritmo

## 3.4 Dataset Iris (Lirios)


![texto alternativo](https://cdn0.ecologiaverde.com/es/posts/4/1/0/tipos_de_lirios_3014_orig.jpg)

El dataset Iris es un dataset clásico de clasificación. Contiene 150 observaciones de 3 clases de flores distintas (50 observaciones por clase). Cada observación tiene 4 features: largo y ancho del sépalo y largo y ancho del pétalo.

Fue introducido por el estadístico y biólogo británico Ronald Fisher en 1936.

Dato importante: si tienen gatos, no tengan lirios cerca. Son sumamente tóxicos.

In [ ]:
from sklearn import datasets
import pandas as pd

iris = datasets.load_iris()
X = iris.data
y = iris.target

df = pd.DataFrame(X, columns=[x.strip(" (cm)") for x in iris.feature_names])
df["species"] = [iris.target_names[x] for x in y]
df

### 3.4.1 Análisis exploratorio

Explorar el dataset. ¿Qué tipo de problema es? ¿Qué tipo de features tiene?

Use `sns.displot` y `sns.pairplot` para visualizar las distribuciones de las variables y las relaciones entre ellas.

¿Qué clase es más fácil de separar de las otras dos? ¿Cuál es más difícil?

### 3.4.2 Split train/test

Utilice la función `train_test_split` de `sklearn.model_selection` para dividir los datos en train y test. Utilice un 66% para train y un 33% para test.

Setee como `random_state` el número 42 (¿para qué sirve?). Usar `stratify` para que los datos se dividan de manera estratificada (¿qué es esto?)


Consultar [la documentación de `train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) para ver cómo se usa

In [ ]:
# Train / test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = None



In [ ]:
df.corr()

### 3.4.3 Entrenamiento de modelos

Entrenar los modelos que ya programamos y predecir sobre los datos de test. Reportar accuracy, precision, recall y F1-score para cada clasificador (pueden usar `sklearn.metrics.classification_report`)

## 3.4.4 Análisis de errores

Calcular la matriz de confusión para el mejor clasificador. ¿Qué clase se confunde más con qué clase? ¿Por qué?

Usar `sklearn.metrics.confusion_matrix`

## 4. KNN

### 4.1 Implementar KNN

Implementar un clasificador KNN. Pasar como parámetro $k$ en el constructor de la clase.

Correr sobre el dataset Iris y reportar accuracy, precision, recall y F1-score. ¿Cómo se compara con los clasificadores anteriores?